In [33]:
%load_ext autoreload
%autoreload 2
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/NLP/project

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/NLP/project


In [27]:
%pip install transformers datasets

     |████████████████████████████████| 290 kB 5.0 MB/s 
     |████████████████████████████████| 243 kB 52.2 MB/s 
     |████████████████████████████████| 132 kB 72.3 MB/s 
     |████████████████████████████████| 1.1 MB 45.9 MB/s 
     |████████████████████████████████| 160 kB 53.0 MB/s 
     |████████████████████████████████| 271 kB 59.6 MB/s 
     |████████████████████████████████| 192 kB 73.6 MB/s 


In [7]:
!nvidia-smi

Wed Nov 17 20:32:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [34]:
!python run_mlm.py --output_dir=models/elonmusk --overwrite_output_dir --model_type=bert --model_name_or_path=bert-base-cased \
  --do_train --train_file=data/elonmusk/train.txt --validation_file=data/elonmusk/valid.txt --do_eval --per_device_train_batch_size=2 \
  --per_device_eval_batch_size=2 --line_by_line --learning_rate=5e-5 --num_train_epochs=5

11/17/2021 21:22:35 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
11/17/2021 21:22:35 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_at

In [35]:
!python run_generation.py \
  --model_type bert \
  --model_name_or_path "models/elonmusk" \
  --length 50 \
  --prompt "<BOS>" \
  --stop_token "<EOS>" \
  --k 50 \
  --num_return_sequences 5

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
11/17/2021 21:31:06 - INFO - __main__ -   Setting is_decoder to True
11/17/2021 21:31:06 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=50, length=50, model_name_or_path='models/elonmusk', model_type='bert', n_gpu=1, no_cuda=False, num_return_sequences=5, p=0.9, padding_text='', prompt='<BOS>', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=1.0, xlm_language='')
=== GENERATED SEQUENCE 1 ===
''''''''''''''''''''''''''''''''
=== GENERATED SEQUENCE 2 ===
b b b b b b b b b b b b b b b b b b b b b b b b b b b b b b b b b b b b 
=== GENERATED SEQUENCE 3 ===
................,,,,,,,,,,,,,,,,
=== GENERATED SEQUENCE 4 ===
'''''''''''''''''''''''''''''''' 
=== GENERATED SEQUENCE 5 ===
..'''''''''''''''''''''''''''''' 


In [8]:
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

import pandas as pd
import seaborn as sns
import numpy as np
import random

import matplotlib.pyplot as plt

from BERT import TweetDataset, train, generate, save_model, load_model, collate_batch
from transformers import BertLMHeadModel,  BertTokenizer, BertConfig, BertLMHeadModel
from transformers import AdamW, __version__ 

from tqdm import tqdm

import nltk
nltk.download('punkt')
torch.cuda.empty_cache()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [9]:
device = torch.device('cuda')
BOS_TOKEN = "<BOS>"
EOS_TOKEN = "<EOS>"
PAD_TOKEN = "<PAD>"
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', bos_token=BOS_TOKEN, eos_token=EOS_TOKEN, pad_token=PAD_TOKEN)


Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
epochs = 5
learning_rate = 5e-4
epsilon = 1e-8
sample_every = 100
batch_size = 8
configuration = BertConfig.from_pretrained("bert-base-cased")
configuration.is_decoder=True

# instantiate the model
model = BertLMHeadModel.from_pretrained("bert-base-cased", config=configuration).to(device)
optimizer = AdamW(model.parameters(), lr=learning_rate, eps=epsilon)

# this step is necessary because I've added some tokens to the embeddings
# otherwise the tokenizer and model tensors won't match up
model.resize_token_embeddings(len(tokenizer))
model.cuda()

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
def train_loss_plot(training_stats):
    # Display floats with two decimal places.
    pd.set_option('precision', 2)
    df_stats = pd.DataFrame(data=training_stats)
    df_stats = df_stats.set_index('epoch')

    # Plot the loss
    sns.set(style='darkgrid')
    sns.set(font_scale=1.5)
    plt.rcParams["figure.figsize"] = (12,6)
    plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
    plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")
    plt.title("Training & Validation Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.xticks([1, 2, 3, 4])
    plt.show()

In [12]:
train_dataset = TweetDataset(tokenizer, 'data/elonmusk/train.txt')
val_dataset = TweetDataset(tokenizer, 'data/elonmusk/test.txt')

11/17/2021 20:32:36 - INFO - Creating features from dataset file at data/elonmusk/train.txt
11/17/2021 20:32:37 - INFO - Creating features from dataset file at data/elonmusk/test.txt


In [13]:
train_dataloader = DataLoader(
    train_dataset,
    sampler=RandomSampler(train_dataset), # Select batches randomly
    batch_size=batch_size,
)

val_dataloader = DataLoader(
    val_dataset,
    sampler=SequentialSampler(val_dataset), # Pull out batches sequentially.
    batch_size=batch_size,
)

In [14]:
training_stats = train(model, device, optimizer, train_dataloader, val_dataloader, tokenizer,
                       epochs=epochs, sample_every=sample_every)

11/17/2021 20:32:37 - INFO - ======== Epoch 1 / 5 ========
11/17/2021 20:32:37 - INFO - Training...


RuntimeError: ignored

In [ ]:
train_loss_plot(training_stats)

In [ ]:
save_model('elonmusk', model, tokenizer)

In [ ]:
model, tokenizer = load_model('elonmusk')
model = model.to(device)

In [ ]:
tweets = generate(model, device, tokenizer, n_sequences=10, max_length=50)
tweets